In [1]:
# Convert Switzerland.geojson to switzerland.json in format 
# [
#   {p: [{x: 1, y: 2}, {x: 2, y: 3}, {x: 3, y: 4}, {x: 1, y: 2}]}, # This is a ring, a sequence of points
#   {p: [{x: 4, y: 5}, {x: 6, y: 7}, {x: 8, y: 9}, {x: 4, y: 5}]}
# ]

In [2]:
import json

In [4]:
with open('./data/switzerland.geojson') as f:
    data = json.load(f)

In [7]:
newdata = []

for f in data['features']:
    for ring in f['geometry']['coordinates']:
        temp = {'p': []}
        for point in ring:
            temp['p'].append({'x':point[0], 'y':point[1]})

        newdata.append(temp)

In [12]:
with open('./results/switzerland.json', 'w') as f:
    f.write(json.dumps(newdata,separators=(',', ':')))